In [1]:
import os
import random
import numpy as np
import pandas as pd
import optuna   
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report,  balanced_accuracy_score
from optuna.visualization import plot_optimization_history
import json
import time

/home/toyotx22/bitcoin_sentiment_forecaster/tf_venv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEED = 43
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"
model_name = 'Multiclass Neural Network'

Score Trend Changes Score

In [3]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [4]:
# Cargar datos
train = pd.read_csv("../../../data/post_cleaning/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/post_cleaning/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values
# Mapear clases (-1,0,1) -> (0,1,2) 
cls_map = {-1:0, 0:1, 1:2}
inv_map = {v:k for k,v in cls_map.items()}
y_train_m = np.vectorize(cls_map.get)(y_train)
y_val_m   = np.vectorize(cls_map.get)(y_val)

OPTIMIZACION CON OPTUNA

In [5]:
def objective(trial):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    # Hiperparámetros a optimizar
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    n_l1 = trial.suggest_int("n_l1", 32, 256, step=32)
    n_l2 = trial.suggest_int("n_l2", 32, 256, step=32)
    n_l3 = trial.suggest_int("n_l3", 16, 128, step=16)
    n_l4 = trial.suggest_int("n_l4", 8, 64, step=8)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    activation = trial.suggest_categorical("activation", ["relu", "tanh", "selu"])
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop", "sgd"])
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])

    # Optimizer
    optimizer = {
        "adam": tf.keras.optimizers.Adam,
        "rmsprop": tf.keras.optimizers.RMSprop,
        "sgd": tf.keras.optimizers.SGD
    }[optimizer_name](learning_rate=learning_rate)

    # Modelo
    model = Sequential([
        tf.keras.Input(shape=(X_train.shape[1],)),
        Dense(n_l1, activation=activation),
        Dropout(dropout_rate),
        Dense(n_l2, activation=activation),
        Dropout(dropout_rate),
        Dense(n_l3, activation=activation),
        Dropout(dropout_rate),
        Dense(n_l4, activation=activation),
        Dropout(dropout_rate),
        Dense(3, activation='linear')  # 3 clases
    ])

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        metrics=["accuracy"]
    )

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True
    )

    # Entrenamiento
    model.fit(
        X_train, y_train_m,
        validation_data=(X_val, y_val_m),
        epochs=60,
        batch_size=batch_size,
        callbacks=[early_stop],
        verbose=0
    )

    # Predicción y evaluación con tu métrica personalizada
    y_val_prob = model.predict(X_val)
    y_val_pred = np.argmax(y_val_prob, axis=1)

    score = trend_changes_true(y_val_m, y_val_pred)
    return score  # Este es el valor que Optuna maximiza

In [6]:
# Crear estudio y medir tiempo de Optuna
t0 = time.perf_counter()

study = optuna.create_study(
    direction="maximize", 
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=400)

opt_duration_sec = time.perf_counter() - t0
n_trials_run = len(study.trials)
print(f"Optuna duró {opt_duration_sec:.2f}s en {n_trials_run} trials")

[I 2025-09-17 17:42:31,923] A new study created in memory with name: no-name-9d103d72-9395-4814-a5d6-4cda739913fb
2025-09-17 17:42:31.984909: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-09-17 17:42:32.296710: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


10/10 [==============================] - 0s 2ms/step


[I 2025-09-17 17:42:34,713] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 2.2139290514335146e-05, 'n_l1': 160, 'n_l2': 64, 'n_l3': 32, 'n_l4': 24, 'dropout_rate': 0.42956874547429885, 'activation': 'relu', 'optimizer': 'sgd', 'batch_size': 256}. Best is trial 0 with value: 0.0.


10/10 [==============================] - 0s 1ms/step


[I 2025-09-17 17:42:43,812] Trial 1 finished with value: 0.0 and parameters: {'learning_rate': 4.603487705829326e-05, 'n_l1': 128, 'n_l2': 96, 'n_l3': 16, 'n_l4': 56, 'dropout_rate': 0.4244695738724307, 'activation': 'relu', 'optimizer': 'rmsprop', 'batch_size': 64}. Best is trial 0 with value: 0.0.


...

10/10 [==============================] - 0s 2ms/step


[I 2025-09-17 18:09:19,644] Trial 398 finished with value: 0.0 and parameters: {'learning_rate': 0.0002819967446257086, 'n_l1': 64, 'n_l2': 96, 'n_l3': 32, 'n_l4': 8, 'dropout_rate': 0.023257924484205147, 'activation': 'selu', 'optimizer': 'adam', 'batch_size': 64}. Best is trial 203 with value: 0.5930232558139535.


10/10 [==============================] - 0s 1ms/step


[I 2025-09-17 18:09:24,968] Trial 399 finished with value: 0.3472222222222222 and parameters: {'learning_rate': 0.00011165189991909266, 'n_l1': 32, 'n_l2': 96, 'n_l3': 32, 'n_l4': 8, 'dropout_rate': 0.0095617519974915, 'activation': 'selu', 'optimizer': 'adam', 'batch_size': 64}. Best is trial 203 with value: 0.5930232558139535.


Optuna duró 1763.02s en 400 trials


In [7]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [8]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'learning_rate': 0.000102413766392232, 'n_l1': 64, 'n_l2': 96, 'n_l3': 32, 'n_l4': 8, 'dropout_rate': 0.008540077597569223, 'activation': 'selu', 'optimizer': 'adam', 'batch_size': 256}
Mejor score de f1-score: 0.5930


GUARDAR EN JSON

In [9]:
# Guardar los mejores hiperparámetros y su valor
history = []
if os.path.exists("best_hyperparams.json"):
    try:
        with open("best_hyperparams.json", "r") as f:
            history = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history = []

# Guardar ambos en un solo diccionario
history.append({
    "params": study.best_params,
    "value": study.best_value
})

with open("best_hyperparams.json", "w") as f:
    json.dump(history, f, indent=2)

CARGAR HIPERPARAMETROS DESDE JSON

In [10]:
# Cargar historial de hiperparámetros y valores
with open("best_hyperparams.json", "r") as f:
    history = json.load(f)

# Escoger el último (más reciente)
best_params = history[-1]["params"]
best_value = history[-1]["value"]

# Para ver todos:
for i, entry in enumerate(history):
    print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# Para escoger uno específico (por índice):
# best_params = history[indice_que_quieras]["params"]
# best_value = history[indice_que_quieras]["value"]

Hiperparámetros #1: {'learning_rate': 0.000102413766392232, 'n_l1': 64, 'n_l2': 96, 'n_l3': 32, 'n_l4': 8, 'dropout_rate': 0.008540077597569223, 'activation': 'selu', 'optimizer': 'adam', 'batch_size': 256}, Valor: 0.5930232558139535
Hiperparámetros #2: {'learning_rate': 0.000102413766392232, 'n_l1': 64, 'n_l2': 96, 'n_l3': 32, 'n_l4': 8, 'dropout_rate': 0.008540077597569223, 'activation': 'selu', 'optimizer': 'adam', 'batch_size': 256}, Valor: 0.5930232558139535


In [11]:
# Carga de librerias con su semilla para garantizar reproducibilidad
import os
os.environ['PYTHONHASHSEED'] = str(SEED)
import random
random.seed(SEED)
import numpy as np
np.random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)

# Modelo final con los mejores hiperparámetros
final_optimizer = {
    "adam": tf.keras.optimizers.Adam,
    "rmsprop": tf.keras.optimizers.RMSprop,
    "sgd": tf.keras.optimizers.SGD
}[best_params["optimizer"]](learning_rate=best_params["learning_rate"])
final_model = Sequential([
    Dense(best_params["n_l1"], activation=best_params["activation"]),
    Dropout(best_params["dropout_rate"]),
    Dense(best_params["n_l2"], activation=best_params["activation"]),
    Dropout(best_params["dropout_rate"]),
    Dense(best_params["n_l3"], activation=best_params["activation"]),
    Dropout(best_params["dropout_rate"]),
    Dense(best_params["n_l4"], activation=best_params["activation"]),
    Dropout(best_params["dropout_rate"]),
    Dense(3, activation='linear')  # 3 clases
])
final_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=final_optimizer,
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

history = final_model.fit(
    X_train, y_train_m,
    validation_data=(X_val, y_val_m),
    epochs=60,
    batch_size=best_params["batch_size"],
    callbacks=[early_stop],
    verbose=0
)
final_model.save("Multiclass_Neural_Network.keras")

In [12]:
# Obtener predicciones finales
y_val_pred = np.argmax(final_model.predict(X_val), axis=1)
print("Multiclass Neural Network Trend Changes Score:\n", trend_changes_score(y_val_m, y_val_pred))

10/10 [==============================] - 0s 2ms/step
Multiclass Neural Network Trend Changes Score:
               precision    recall  f1-score   support

       False     0.8821    0.7854    0.8309       219
        True     0.5204    0.6892    0.5930        74

    accuracy                         0.7611       293
   macro avg     0.7012    0.7373    0.7120       293
weighted avg     0.7907    0.7611    0.7708       293



In [13]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val_m, y_val_pred, digits=4)
print("Multiclass Neural Network Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val_m, y_val_pred))

Multiclass Neural Network Report:
               precision    recall  f1-score   support

           0     0.7475    0.8043    0.7749        92
           1     0.3488    0.3061    0.3261        49
           2     0.8355    0.8301    0.8328       153

    accuracy                         0.7347       294
   macro avg     0.6439    0.6468    0.6446       294
weighted avg     0.7269    0.7347    0.7302       294

Balanced accuracy: 0.6468452115145576


In [ ]:
# --- Exportar y Comparar Métricas de Modelos (Validación) ---

# 1. Definir el nombre del modelo actual y el archivo de salida
output_file = '../../../score_models/model_comparison_metrics.csv'

# 2. Calcular el reporte de clasificación estándar
# Usamos y_val_m y y_val_pred que están en la misma escala (0,1,2)
report_dict = classification_report(y_val_m, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score = report_dict['macro avg']['f1-score']


# 3. Calcular el reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val_m, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar las nuevas métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score,
    "optuna_duration_sec": opt_duration_sec,
    "n_trials": n_trials_run
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
try:
    # Intentar cargar el archivo existente
    comparison_df = pd.read_csv(output_file, index_col='model')
    # Si existe, actualizar o añadir la fila para el modelo actual
    comparison_df.loc[model_name] = new_metrics
except FileNotFoundError:
    # Si no existe, crear un DataFrame nuevo directamente con los datos actuales
    comparison_df = pd.DataFrame([new_metrics], index=[model_name])

# Guardar el DataFrame actualizado en el CSV
comparison_df.to_csv(output_file, index_label='model')

: 